# <center> Final Project </center>
## <center> Group 3 </center>

## Elia Rezaeian

_________________________________________________________________________________________________________________________________________________

### SECTION3 : Train Model-Stacking Method
_________________________________________________________________________________________________________________________________________________


#### import the library

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


/var/folders/zy/_h1xy46d69jg35qmf_0x_vnw0000gn/T/ipykernel_26683/1064586460.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('Section1&2(Clusters).csv')

In [3]:
df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,Cluster,Bankrupt?,Predicted_Cluster
0,2.658210,-1.400730,-0.381088,0.452337,0.858035,-0.012784,-0.381166,0.117564,0.221323,1,0,1
1,-1.428676,0.461506,-1.030601,-0.234852,-0.029953,-0.033227,-0.566027,0.076135,0.440510,5,0,5
2,-2.775424,1.442919,-0.711948,0.144689,-0.152528,0.164080,0.258936,-0.555779,-0.286176,5,0,5
3,2.403655,4.045684,0.326334,-1.356795,-0.638412,-0.146502,-0.058012,0.546713,-0.348422,0,0,0
4,0.865429,0.283041,0.526507,-0.195726,-0.437119,-0.979682,-0.145639,-0.450059,0.824183,0,0,0


Here we split the data into X and Y and then I select companies in clusters 0 and 1 

In [4]:
X = df.iloc[:,:-3]
y = df.iloc[:,-2]

In [5]:
X_0 = X[df.iloc[:,-1] == 0] 
X_1 = X[df.iloc[:,-1] == 1]

y_0 = y[df.iloc[:,-1] == 0]
y_1 = y[df.iloc[:,-1] == 1]

#### Here I wrote a function for models Tuning that we can apply on differtent data and scoring using Stratified Kfold and Gridsearch CV

In [6]:
# write a function that have all base models with tuning their hyperparameters using grid search

def get_models():
    models = dict()
    models['Logistic'] = LogisticRegression(solver='liblinear',random_state=42)
    models['SVM'] = SVC(random_state=42)
    models['KNN'] = KNeighborsClassifier()
    models['RandomForest'] = RandomForestClassifier(random_state=42)
    models['xgboost'] = xgb.XGBClassifier(random_state=42)
    models['NeuralNetwork'] = MLPClassifier(random_state=42)
    models['DecisionTree'] = DecisionTreeClassifier(random_state=42)
    models['AdaBoost'] = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=42),random_state=42)
    models['GradientBoosting'] = GradientBoostingClassifier(random_state=42)
    models['NaiveBayes'] = GaussianNB()
    models['Bagging'] = BaggingClassifier(random_state=42)
    
    return models

def get_params():
    
    params = dict()
    params['Logistic'] =  { 'penalty': ['l1', 'l2', 'elasticnet', 'none'],'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'class_weight': ['balanced', None], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],'max_iter': [100, 200, 300, 400, 500]}
    
    params['SVM'] = {'C': [ 0.01, 0.1, 1 ], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'degree': [2, 3, 4, 5],
    'gamma': ['scale', 'auto'], 'class_weight': ['balanced']}
    
    params['KNN'] = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40, 50],'p': [1, 2]}
    
    params['RandomForest'] = param_grid = {'n_estimators': [100, 200, 300, 400], 'max_depth': [None, 10, 20, 30, 40],   
    'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4],'bootstrap': [True, False],'class_weight': ['balanced', 'balanced_subsample'],  
    'max_features': ['auto', 'sqrt', 'log2']}
    
    params['xgboost'] = {'max_depth': [3, 4, 5, 6],'min_child_weight': [1, 2, 3],'gamma': [0, 0.1, 0.2, 0.3],'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],'learning_rate': [0.01, 0.1, 0.2, 0.3],'n_estimators': [100, 200, 300]}
    
    params['NeuralNetwork'] = {'hidden_layer_sizes': [(100,), (50, 50), (100, 100), (50, 50, 50)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],'solver': ['lbfgs', 'sgd', 'adam'],'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],'max_iter': [200, 300, 400]}
    
    params['DecisionTree'] = {'max_depth': [None, 10, 20, 30, 50], 'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],'criterion': ['gini', 'entropy']}
    
    params['AdaBoost'] = {'n_estimators': [50, 100, 200, 300],'learning_rate': [0.01, 0.1, 1, 10],
    'base_estimator__max_depth': [1, 2, 3]}
    
    params['GradientBoosting'] = {'n_estimators': [100, 200, 300, 400],'learning_rate': [0.01, 0.1, 0.5, 1],
    'max_depth': [3, 5, 8],'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4],'max_features': ['sqrt', 'log2', None]}
    
    params['NaiveBayes'] = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}
    
    params['Bagging'] = {'n_estimators': [10, 50, 100, 200],'max_samples': [0.5, 0.7, 1.0], 'max_features': [0.5, 0.7, 1.0],  
    'bootstrap': [True, False],'bootstrap_features': [True, False]}
    
    return params

from sklearn.model_selection import GridSearchCV, StratifiedKFold

def get_best_model(X, y, models, params, score):
    best_model = dict()
    for key in models.keys():
        model = models[key]
        param = params[key]
        cv = StratifiedKFold(n_splits=3, shuffle = True, random_state=42)
        grid_search = GridSearchCV(estimator=model, param_grid=param, n_jobs=-1, cv=cv, scoring = score)
        grid_result = grid_search.fit(X, y)
        best_model[key] = grid_result.best_estimator_
        # Print the best model and best score for each type of model
        print(f"Best Model for {key}: {grid_result.best_estimator_}")
        print(f"Best Parameters {key}: {grid_result.best_params_}")
        print(f"Best Score for {key}: {grid_result.best_score_:.4f}\n")
    return best_model

Hyper parameter tuning for cluster 1 

In [7]:
#apply on cluster 0

models_0 = get_models()
params_0 = get_params()
best_model_0 = get_best_model(X_0, y_0, models_0, params_0, 'f1')

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.         ...        nan 0.03703704 0.03703704]
  category=UserWarning
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Best Model for Logistic: LogisticRegression(C=0.1, class_weight='balanced', random_state=42,
                   solver='saga')
Best Parameters Logistic: {'C': 0.1, 'class_weight': 'balanced', 'max_iter': 100, 'penalty': 'l2', 'solver': 'saga'}
Best Score for Logistic: 0.2413

Best Model for SVM: SVC(C=0.01, class_weight='balanced', degree=5, kernel='poly', random_state=42)
Best Parameters SVM: {'C': 0.01, 'class_weight': 'balanced', 'degree': 5, 'gamma': 'scale', 'kernel': 'poly'}
Best Score for SVM: 0.2405

Best Model for KNN: KNeighborsClassifier(leaf_size=10, n_neighbors=9, p=1)
Best Parameters KNN: {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 9, 'p': 1, 'weights': 'uniform'}
Best Score for KNN: 0.0784

Best Model for RandomForest: RandomForestClassifier(class_weight='balanced_subsample', max_depth=10,
                       min_samples_leaf=4, min_samples_split=10,
                       random_state=42)
Best Parameters RandomForest: {'bootstrap': True, 'class_weight': 'ba

Here I selected 3 best model from the previous cell and apply a meta learner using logistic regression. 

In [20]:
# Cluster 0 
# Best models for cluster 0 which we can use in our meta model
LR_0 = best_model_0['Logistic']
SVM_0 = best_model_0['SVM']
NN_0 = best_model_0['NeuralNetwork']

model_0_1 = LR_0.fit(X_0,y_0)
model_0_2 = SVM_0.fit(X_0,y_0)
model_0_3 = NN_0.fit(X_0,y_0)

LR_pred_0 = model_0_1.predict(X_0)
SVM_pred_0 = model_0_2.predict(X_0)
NN_pred_0 = model_0_3.predict(X_0)

print("Logistic Regression confustion matrix:\n",confusion_matrix(y_0,LR_pred_0 ))
print("SVM confustion matrix:\n",confusion_matrix(y_0,SVM_pred_0 ))
print("NN confustion matrix:\n",confusion_matrix(y_0,NN_pred_0 ))

X_Stack_0 = pd.DataFrame()
X_Stack_0['NN'] = NN_pred_0
X_Stack_0['SVM'] = SVM_pred_0
X_Stack_0['LR'] = LR_pred_0

meta_learner = LogisticRegression(class_weight='balanced', random_state=42)
meta_learner.fit(X_Stack_0, y_0)

meta_learner_pred = meta_learner.predict(X_Stack_0)
meta_learner_classification_report = classification_report(y_0, meta_learner_pred)
print("Meta learner Classification report:\n",meta_learner_classification_report)
meta_learner_cm = confusion_matrix(y_0, meta_learner_pred)
print("Meta learner Confusion Matrix:\n",meta_learner_cm)
accuracy_score_meta1 = accuracy_score(y_0, meta_learner_pred)
print("Meta learner Accuracy:\n", accuracy_score_meta)


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Logistic Regression confustion matrix:
 [[908 275]
 [ 12  36]]
SVM confustion matrix:
 [[1047  136]
 [  18   30]]
NN confustion matrix:
 [[1183    0]
 [   0   48]]
Meta learner Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1183
           1       1.00      1.00      1.00        48

    accuracy                           1.00      1231
   macro avg       1.00      1.00      1.00      1231
weighted avg       1.00      1.00      1.00      1231

Meta learner Confusion Matrix:
 [[1183    0]
 [   0   48]]
Meta learner Accuracy:
 1.0


* The TT/(TT+TF) for logistic regression is = > 75%
* The TT/(TT+TF) for SVM is = > 62%
* The TT/(TT+TF) Neural Network is = > 100% 

when I use meta learner in the for ensemble stacking, I got 100% accuracy. 


here I tune the hyperparameters on Cluster 1 

In [9]:
#apply on cluster 1

models_1 = get_models()
params_1 = get_params()
best_model_1 = get_best_model(X_1, y_1, models_1, params_1, 'f1')

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [nan nan  0. ... nan  0.  0.]
  category=UserWarning
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Best Model for Logistic: LogisticRegression(C=0.01, class_weight='balanced', max_iter=300, penalty='l1',
                   random_state=42, solver='saga')
Best Parameters Logistic: {'C': 0.01, 'class_weight': 'balanced', 'max_iter': 300, 'penalty': 'l1', 'solver': 'saga'}
Best Score for Logistic: 0.1752

Best Model for SVM: SVC(C=0.1, class_weight='balanced', degree=5, kernel='poly', random_state=42)
Best Parameters SVM: {'C': 0.1, 'class_weight': 'balanced', 'degree': 5, 'gamma': 'scale', 'kernel': 'poly'}
Best Score for SVM: 0.2000

Best Model for KNN: KNeighborsClassifier(leaf_size=10, n_neighbors=3, p=1)
Best Parameters KNN: {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}
Best Score for KNN: 0.0000

Best Model for RandomForest: RandomForestClassifier(class_weight='balanced', random_state=42)
Best Parameters RandomForest: {'bootstrap': True, 'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samp

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Best Model for NeuralNetwork: MLPClassifier(activation='tanh', max_iter=300, random_state=42)
Best Parameters NeuralNetwork: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'max_iter': 300, 'solver': 'adam'}
Best Score for NeuralNetwork: 0.1667

Best Model for DecisionTree: DecisionTreeClassifier(random_state=42)
Best Parameters DecisionTree: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Score for DecisionTree: 0.0000

Best Model for AdaBoost: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                         random_state=42),
                   learning_rate=10, random_state=42)
Best Parameters AdaBoost: {'base_estimator__max_depth': 3, 'learning_rate': 10, 'n_estimators': 50}
Best Score for AdaBoost: 0.0558

Best Model for GradientBoosting: GradientBoostingClassifier(learning_rate=0.5, max_features='sqrt',
                      

Here I selected 3 best model from the previous cell and apply a meta learner using logistic regression. 

In [18]:
# Best models for cluster 0 which we can use in our meta model
LR_1 = best_model_0['Logistic']
SVM_1 = best_model_0['SVM']
NN_0 = best_model_0['NeuralNetwork']

model_1_1 = LR_0.fit(X_1,y_1)
model_1_2 = SVM_0.fit(X_1,y_1)
model_1_3 = NN_0.fit(X_1,y_1)

LR_pred_1 = model_1_1.predict(X_1)
SVM_pred_1 = model_1_2.predict(X_1)
NN_pred_1 = model_1_3.predict(X_1)

print("Logistic Regression confustion matrix:\n",confusion_matrix(y_1,LR_pred_1 ))
print("SVM confustion matrix:\n",confusion_matrix(y_1,SVM_pred_1 ))
print("NN confustion matrix:\n",confusion_matrix(y_1,NN_pred_1 ))

X_Stack_1 = pd.DataFrame()
X_Stack_1['NN'] = NN_pred_1
X_Stack_1['SVM'] = SVM_pred_1
X_Stack_1['LR'] = LR_pred_1

meta_learner1 = LogisticRegression(class_weight='balanced', random_state=42)
meta_learner1.fit(X_Stack_1, y_1)

meta_learner_pred1 = meta_learner1.predict(X_Stack_1)
meta_learner_classification_report1 = classification_report(y_1, meta_learner_pred1)
print("Meta learner Classification report:\n",meta_learner_classification_report1)
meta_learner_cm1 = confusion_matrix(y_1, meta_learner_pred1)
print("Meta learner Confusion Matrix:\n",meta_learner_cm1)
accuracy_score_meta1 = accuracy_score(y_1, meta_learner_pred1)
print("Meta learner Accuracy:\n", accuracy_score_meta1)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Logistic Regression confustion matrix:
 [[317 218]
 [  2   8]]
SVM confustion matrix:
 [[514  21]
 [  4   6]]
NN confustion matrix:
 [[535   0]
 [  0  10]]
Meta learner Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       535
           1       1.00      1.00      1.00        10

    accuracy                           1.00       545
   macro avg       1.00      1.00      1.00       545
weighted avg       1.00      1.00      1.00       545

Meta learner Confusion Matrix:
 [[535   0]
 [  0  10]]
Meta learner Accuracy:
 1.0


* The TT/(TT+TF) for logistic regression is = > 80%
* The TT/(TT+TF) for SVM is = > 60%
* The TT/(TT+TF) Neural Network is = > 100% 

when I use meta learner in the for ensemble stacking, I got 100% accuracy. 


_________________________________________________________________________________________________________________________________________________
### SECTION4 : K_Fold Cross Validation on the Whole Dataset
_________________________________________________________________________________________________________________________________________________

Import the libraries

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score


In [ ]:
df = pd.read_csv('Section1&2(Clusters).csv')
df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,Cluster,Bankrupt?,Predicted_Cluster
0,2.658210,-1.400730,-0.381088,0.452337,0.858035,-0.012784,-0.381166,0.117564,0.221323,1,0,1
1,-1.428676,0.461506,-1.030601,-0.234852,-0.029953,-0.033227,-0.566027,0.076135,0.440510,5,0,5
2,-2.775424,1.442919,-0.711948,0.144689,-0.152528,0.164080,0.258936,-0.555779,-0.286176,5,0,5
3,2.403655,4.045684,0.326334,-1.356795,-0.638412,-0.146502,-0.058012,0.546713,-0.348422,0,0,0
4,0.865429,0.283041,0.526507,-0.195726,-0.437119,-0.979682,-0.145639,-0.450059,0.824183,0,0,0


In [ ]:
X = df.iloc[:,0:9]
y = df.iloc[:,10]   

here I first run all models without tuning on the whole dataset and got results

In [ ]:
result_whole_data1 = {}
RANDOM_STATE = 42
# Initialize the classifier
models1 = {
    'LogisticRegression': LogisticRegression(solver='liblinear', random_state=42),
    'SVC': SVC(random_state=42),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'RandomForestClassifier': RandomForestClassifier(random_state=42),
    'XGBClassifier': xgb.XGBClassifier(random_state=42),
    'MLPClassifier': MLPClassifier(random_state=42),
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
    'AdaBoostClassifier': AdaBoostClassifier(random_state=42),
    'GradientBoostingClassifier': GradientBoostingClassifier(random_state=42),
    'GaussianNB': GaussianNB(),
    'BaggingClassifier': BaggingClassifier(random_state=42)
}
# Use stratified kfold
skf = StratifiedKFold(n_splits=5, shuffle=True ,random_state=RANDOM_STATE)

for model_name, model in models1.items():
    recall_scores = cross_val_score(model, X, y, cv=skf, scoring='recall')
    mean_scores = recall_scores.mean()
    # find cm
    y_pred = cross_val_predict(model, X, y, cv=skf)
    cm = confusion_matrix(y, y_pred)
    accuracy = accuracy_score(y, y_pred)
    result_whole_data1[model_name] = (cm, mean_scores, accuracy)

# print("Results for whole data:")
# print(result_whole_data)

/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/elia/Library/

The results here shows that while the accuracy is high but they cannot predict the Instanses with label 1 well. 

In [ ]:

print("LogisticRegression confustion matrix:\n", result_whole_data1['LogisticRegression'][0])
print("TT/(TT+TF) = ", result_whole_data1['LogisticRegression'][1])
print("Accuracy = ", result_whole_data1['LogisticRegression'][2])
print("-------------------------------------")
print("SVC confustion matrix:\n", result_whole_data1['SVC'][0])
print("TT/(TT+TF) = ", result_whole_data1['SVC'][1])
print("Accuracy = ", result_whole_data1['SVC'][2])
print("-------------------------------------")
print("KNeighborsClassifier confustion matrix:\n", result_whole_data1['KNeighborsClassifier'][0])
print("TT/(TT+TF) = ", result_whole_data1['KNeighborsClassifier'][1])
print("Accuracy = ", result_whole_data1['KNeighborsClassifier'][2])
print("-------------------------------------")
print("RandomForestClassifier confustion matrix:\n", result_whole_data1['RandomForestClassifier'][0])
print("TT/(TT+TF) = ", result_whole_data1['RandomForestClassifier'][1])
print("Accuracy = ", result_whole_data1['RandomForestClassifier'][2])
print("-------------------------------------")
print("XGBClassifier confustion matrix:\n", result_whole_data1['XGBClassifier'][0])
print("TT/(TT+TF) = ", result_whole_data1['XGBClassifier'][1])
print("Accuracy = ", result_whole_data1['XGBClassifier'][2])
print("-------------------------------------")
print("MLPClassifier confustion matrix:\n", result_whole_data1['MLPClassifier'][0])
print("TT/(TT+TF) = ", result_whole_data1['MLPClassifier'][1])
print("Accuracy = ", result_whole_data1['MLPClassifier'][2])
print("-------------------------------------")
print("DecisionTreeClassifier confustion matrix:\n", result_whole_data1['DecisionTreeClassifier'][0])
print("TT/(TT+TF) = ", result_whole_data1['DecisionTreeClassifier'][1])
print("Accuracy = ", result_whole_data1['DecisionTreeClassifier'][2])
print("-------------------------------------")
print("AdaBoostClassifier confustion matrix:\n", result_whole_data1['AdaBoostClassifier'][0])
print("TT/(TT+TF) = ", result_whole_data1['AdaBoostClassifier'][1])
print("Accuracy = ", result_whole_data1['AdaBoostClassifier'][2])
print("-------------------------------------")
print("GradientBoostingClassifier confustion matrix:\n", result_whole_data1['GradientBoostingClassifier'][0])
print("TT/(TT+TF) = ", result_whole_data1['GradientBoostingClassifier'][1])
print("Accuracy = ", result_whole_data1['GradientBoostingClassifier'][2])
print("-------------------------------------")
print("GaussianNB confustion matrix:\n", result_whole_data1['GaussianNB'][0])
print("TT/(TT+TF) = ", result_whole_data1['GaussianNB'][1])
print("Accuracy = ", result_whole_data1['GaussianNB'][2])
print("-------------------------------------")
print("BaggingClassifier confustion matrix:\n", result_whole_data1['BaggingClassifier'][0])
print("TT/(TT+TF) = ", result_whole_data1['BaggingClassifier'][1])
print("Accuracy = ", result_whole_data1['BaggingClassifier'][2])
print("-------------------------------------")

LogisticRegression confustion matrix:
 [[5582   27]
 [ 162   36]]
TT/(TT+TF) =  0.18166666666666667
Accuracy =  0.9674530738763562
-------------------------------------
SVC confustion matrix:
 [[5604    5]
 [ 188   10]]
TT/(TT+TF) =  0.05051282051282051
Accuracy =  0.9667642500430514
-------------------------------------
KNeighborsClassifier confustion matrix:
 [[5558   51]
 [ 151   47]]
TT/(TT+TF) =  0.23756410256410257
Accuracy =  0.965214396418116
-------------------------------------
RandomForestClassifier confustion matrix:
 [[5583   26]
 [ 159   39]]
TT/(TT+TF) =  0.1971794871794872
Accuracy =  0.9681418977096607
-------------------------------------
XGBClassifier confustion matrix:
 [[5562   47]
 [ 150   48]]
TT/(TT+TF) =  0.2421794871794872
Accuracy =  0.9660754262097468
-------------------------------------
MLPClassifier confustion matrix:
 [[5575   34]
 [ 152   46]]
TT/(TT+TF) =  0.23230769230769227
Accuracy =  0.9679696917513346
-------------------------------------
Decision

So here we tried to tune the hyperparameter of all models 

In [ ]:
# write a function that have all base models with tuning their hyperparameters using grid search

def get_models():
    models = dict()
    models['Logistic'] = LogisticRegression(solver='liblinear',random_state=42)
    models['SVM'] = SVC(random_state=42)
    models['KNN'] = KNeighborsClassifier()
    models['RandomForest'] = RandomForestClassifier(random_state=42)
    models['xgboost'] = xgb.XGBClassifier(random_state=42)
    models['NeuralNetwork'] = MLPClassifier(random_state=42)
    models['AdaBoost'] = AdaBoostClassifier(random_state=42)
    models['gradiantboost'] = GradientBoostingClassifier(random_state=42)
    models['Bagging'] = BaggingClassifier(random_state=42)

    
    return models

def get_params():
    
    params = dict()
    params['Logistic'] =  { 'penalty': ['l1', 'l2', 'elasticnet', 'none'],'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'class_weight': ['balanced', None], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],'max_iter': [100, 200, 300, 400, 500]}
    
    params['SVM'] = {'C': [ 0.01, 0.1, 1 ], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'degree': [2, 3, 4, 5],
    'gamma': ['scale', 'auto'], 'class_weight': ['balanced']}
    
    params['KNN'] = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40, 50],'p': [1, 2]}
    
    params['RandomForest'] = param_grid = {'n_estimators': [100, 200, 300, 400], 'max_depth': [None, 10, 20, 30, 40],   
    'min_samples_split': [2, 5, 10],'min_samples_leaf': [1, 2, 4],'bootstrap': [True],'class_weight': ['balanced'],  
    'max_features': ['auto', 'sqrt', 'log2']}
    
    params['NeuralNetwork'] = {'hidden_layer_sizes': [(100,), (50, 50), (100, 100), (50, 50, 50)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],'solver': ['lbfgs', 'sgd', 'adam'],'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],'max_iter': [200, 300, 400]}
    
    params['xgboost'] = {'max_depth': [3, 4, 5, 6],'min_child_weight': [1, 2, 3],'gamma': [0, 0.1, 0.2, 0.3],'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],'learning_rate': [0.01, 0.1, 0.2, 0.3],'n_estimators': [100, 200, 300]}
    
    params['AdaBoost'] = {'n_estimators': [50, 100, 200, 300],'learning_rate': [0.01, 0.1, 1, 10],'algorithm': ['SAMME', 'SAMME.R']} 
  
    params['gradiantboost'] = {'n_estimators': [50, 100, 200, 300],'learning_rate': [0.01, 0.1, 1, 10],'max_depth': [3, 4, 5, 6]}
    
    params['Bagging'] = {'n_estimators': [10, 50, 100, 200],'max_samples': [0.5, 1.0],'max_features': [0.5, 1.0]}   
        
    return params

from sklearn.model_selection import GridSearchCV, StratifiedKFold

def get_best_model(X, y, models, params, score):
    best_model = dict()
    for key in models.keys():
        model = models[key]
        param = params[key]
        cv = StratifiedKFold(n_splits=3, shuffle= True, random_state=42)
        grid_search = GridSearchCV(estimator=model, param_grid=param, n_jobs=-1, cv=cv, scoring = score)
        grid_result = grid_search.fit(X, y)
        best_model[key] = grid_result.best_estimator_
        # Print the best model and best score for each type of model
        print(f"Best Model for {key}: {grid_result.best_estimator_}")
        print(f"Best Parameters {key}: {grid_result.best_params_}")
        print(f"Best Score for {key}: {grid_result.best_score_:.4f}\n")
    return best_model


In [ ]:
models = get_models()
params = get_params()
best_model= get_best_model(X, y, models, params, 'f1')

/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to ci

Best Model for AdaBoost: AdaBoostClassifier(learning_rate=1, n_estimators=200, random_state=42)
Best Parameters AdaBoost: {'algorithm': 'SAMME.R', 'learning_rate': 1, 'n_estimators': 200}
Best Score for AdaBoost: 0.3627

Best Model for gradiantboost: GradientBoostingClassifier(learning_rate=1, n_estimators=50, random_state=42)
Best Parameters gradiantboost: {'learning_rate': 1, 'max_depth': 3, 'n_estimators': 50}
Best Score for gradiantboost: 0.3485

Best Model for Bagging: BaggingClassifier(max_samples=0.5, n_estimators=200, random_state=42)
Best Parameters Bagging: {'max_features': 1.0, 'max_samples': 0.5, 'n_estimators': 200}
Best Score for Bagging: 0.2845



so I used the tuned models for the k fold cross validation and got much better results here 

In [9]:
# Use kfold on whole data with random forest, XGboost and Bagging
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, make_scorer
result_whole_data = {}
RANDOM_STATE = 42
# Initialize the classifier
models = {
    'Random Forest': RandomForestClassifier(class_weight='balanced', min_samples_leaf=4,min_samples_split=10, n_estimators=200, random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=RANDOM_STATE, colsample_bytree= 0.9, gamma =0.2, learning_rate= 0.3, max_depth = 4, min_child_weight= 3, n_estimators= 300, subsample= 0.8),
    'Bagging': BaggingClassifier(random_state=RANDOM_STATE),
    'Logistic Regression': LogisticRegression(C=1, penalty='l1', random_state=42, solver='liblinear'),
    "SVM" : SVC(C=0.01, class_weight='balanced', degree=5, kernel='poly', random_state=42),
    'KNN': KNeighborsClassifier(leaf_size=10, p=1, weights='distance'),
    'NN':MLPClassifier(activation = 'relu', alpha= 0.0001, hidden_layer_sizes= (100,), learning_rate= 'constant', max_iter= 400, solver= 'adam', random_state = 42),
    'AdaBoost' : AdaBoostClassifier(algorithm='SAMME.R', learning_rate=1, n_estimators=50, random_state=42),
    'GradientBoost': GradientBoostingClassifier(learning_rate=1, max_depth=3, n_estimators=50, random_state=42),
    'Bagging': BaggingClassifier(max_features=1, max_samples=0.5, n_estimators=200, random_state=42)
}
# Use stratified kfold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

for model_name, model in models.items():
    recall_scores = cross_val_score(model, X, y, cv=skf, scoring='recall')
    mean_scores = recall_scores.mean()
    # find cm
    y_pred = cross_val_predict(model, X, y, cv=skf)
    cm = confusion_matrix(y, y_pred)
    acc = accuracy_score(y, y_pred)
    result_whole_data[model_name] = (cm, mean_scores, acc)

# print("Results for whole data:")
# print(result_whole_data)

/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/elia/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/elia/Library/

In [10]:
print("Random Forest confustion matrix:\n", result_whole_data['Random Forest'][0])
print("TT/(TT+TF) = ", result_whole_data['Random Forest'][1])
print('accuracy:', result_whole_data['Random Forest'][2])
print("------------------------------------")
print("XGBoost confustion matrix:\n", result_whole_data['XGBoost'][0])
print("TT/(TT+TF) = ", result_whole_data['XGBoost'][1])
print('accuracy:', result_whole_data['XGBoost'][2])
print("------------------------------------")
print("Bagging confustion matrix:\n", result_whole_data['Bagging'][0])
print("TT/(TT+TF) = ", result_whole_data['Bagging'][1])
print('accuracy:', result_whole_data['Bagging'][2])
print("------------------------------------")
print("Logistic Regression confustion matrix:\n", result_whole_data['Logistic Regression'][0])
print("TT/(TT+TF) = ", result_whole_data['Logistic Regression'][1])
print('accuracy:', result_whole_data['Logistic Regression'][2])
print("------------------------------------")
print("SVM confustion matrix:\n", result_whole_data['SVM'][0])
print("TT/(TT+TF) = ", result_whole_data['SVM'][1])
print('accuracy:', result_whole_data['SVM'][2])
print("------------------------------------")
print("KNN confustion matrix:\n", result_whole_data['KNN'][0])
print("TT/(TT+TF) = ", result_whole_data['KNN'][1])
print('accuracy:', result_whole_data['KNN'][2])
print("------------------------------------")
print("NN confustion matrix:\n", result_whole_data['NN'][0])
print("TT/(TT+TF) = ", result_whole_data['NN'][1])
print('accuracy:', result_whole_data['NN'][2])
print("------------------------------------")
print("AdaBoost confustion matrix:\n", result_whole_data['AdaBoost'][0])
print("TT/(TT+TF) = ", result_whole_data['AdaBoost'][1])
print('accuracy:', result_whole_data['AdaBoost'][2])
print("------------------------------------")
print("GradientBoost confustion matrix:\n", result_whole_data['GradientBoost'][0])
print("TT/(TT+TF) = ", result_whole_data['GradientBoost'][1])
print('accuracy:', result_whole_data['GradientBoost'][2])
print("------------------------------------")
print("Bagging confustion matrix:\n", result_whole_data['Bagging'][0])
print("TT/(TT+TF) = ", result_whole_data['Bagging'][1])
print('accuracy:', result_whole_data['Bagging'][2])


Random Forest confustion matrix:
 [[5494  115]
 [ 116   82]]
TT/(TT+TF) =  0.4142307692307693
accuracy: 0.9602204236266575
------------------------------------
XGBoost confustion matrix:
 [[5554   55]
 [ 153   45]]
TT/(TT+TF) =  0.22717948717948716
accuracy: 0.9641811606681591
------------------------------------
Bagging confustion matrix:
 [[5609    0]
 [ 198    0]]
TT/(TT+TF) =  0.0
accuracy: 0.9659032202514207
------------------------------------
Logistic Regression confustion matrix:
 [[5582   27]
 [ 162   36]]
TT/(TT+TF) =  0.18166666666666667
accuracy: 0.9674530738763562
------------------------------------
SVM confustion matrix:
 [[5191  418]
 [  49  149]]
TT/(TT+TF) =  0.7521794871794871
accuracy: 0.9195798174616842
------------------------------------
KNN confustion matrix:
 [[5560   49]
 [ 156   42]]
TT/(TT+TF) =  0.21217948717948723
accuracy: 0.9646977785431375
------------------------------------
NN confustion matrix:
 [[5553   56]
 [ 141   57]]
TT/(TT+TF) =  0.287820512820

Here as you can see, SVM has the best results. 

*SVM confustion matrix:\

[[5191      418]\
[  49       149]]

* TT/(TT+TF) =  0.7521794871794871
* accuracy: 0.9195798174616842